<a href="https://colab.research.google.com/github/tap-o-chek/Multiple-processing-course/blob/main/labs/MP_Lesson_8_%D0%96%D0%B8%D0%B4%D0%B5%D0%BB%D0%B5%D0%B2%D0%B0_%D0%A1%D0%BE%D1%84%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA



```python
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy
numpy.random.seed(1729)
a = numpy.random.randn(4,4)

a = a.astype(numpy.float32)
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)
```



In [1]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=7205e1929f6a0cbc31c75944e1e6c9282864df504e49bb3c26ee1cd243e6a90d
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [2]:
import numpy
numpy.random.seed(1729)
a = numpy.random.randn(4,4)

In [3]:
a = a.astype(numpy.float32)

In [4]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [5]:
cuda.memcpy_htod(a_gpu, a)

In [6]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [7]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [8]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[-1.3746789  -1.6419895   3.3047218  -1.1505861 ]
 [ 2.1979356   1.8518921  -1.9868276  -1.7164422 ]
 [ 0.14977352  1.058711    0.2419031  -0.44884723]
 [-3.113357    0.11188176  0.32294306 -4.2692833 ]]
[[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]


In [9]:
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)

In [10]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [11]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

In [12]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [13]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

[[-0.5257741   0.447586    1.5295702   0.39143866]
 [ 3.3902154  -2.549993   -1.6982892  -0.2884755 ]
 [-1.2885619  -1.2470585  -1.3161025   1.1179583 ]
 [-2.862448   -0.38533747 -0.0221602  -0.24921691]]
[[ 0.10967004  0.44301215  0.39626622  0.2497974 ]
 [ 1.2984973  -1.2804337  -0.97546583 -0.26908663]
 [-1.1057384  -0.1279927  -0.61782736 -0.98912627]
 [-2.8598924  -0.7943475  -0.30579695  1.7006376 ]]
[[-0.63544416  0.00457387  1.133304    0.14164127]
 [ 2.091718   -1.2695593  -0.7228234  -0.01938889]
 [-0.18282357 -1.1190658  -0.6982751   2.1070845 ]
 [-0.0025556   0.40901005  0.28363675 -1.9498545 ]]


# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices:
    1. Assume square matrices, with dimensions < 1024
    2. Assume square matrices, with dimensions > 1024
    3. Assume non-square matrices, with dimensions > 1024

### Task 1
Write a cuda kernel to find the elementwise square of a matrix

In [14]:
numpy.random.seed(1729)
matrix = numpy.random.randn(4, 4)
matrix = matrix.astype(numpy.float32)

matrix_gpu = cuda.mem_alloc(matrix.nbytes)
cuda.memcpy_htod(matrix_gpu, matrix)

# CUDA-ядро нахождения квадрата элементов матрицы
mod_square = SourceModule("""
__global__ void square_matrix(float *matrix)
{
    int idx_x = threadIdx.x + blockIdx.x * blockDim.x;
    int idx_y = threadIdx.y + blockIdx.y * blockDim.y;
    int idx = idx_y * blockDim.x * gridDim.x + idx_x;

    matrix[idx] = matrix[idx] * matrix[idx];
}
""")

square_func = mod_square.get_function("square_matrix")

block_size = (4, 4, 1)
grid_size = (1, 1)
square_func(matrix_gpu, block=block_size, grid=grid_size)

result = numpy.empty_like(matrix)
cuda.memcpy_dtoh(result, matrix_gpu)

print("Original matrix:")
print(matrix)
print("\nSquared matrix:")
print(result)

Original matrix:
[[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]

Squared matrix:
[[4.7243547e-01 6.7403233e-01 2.7302966e+00 3.3096212e-01]
 [1.2077302e+00 8.5737610e-01 9.8687100e-01 7.3654348e-01]
 [5.6080269e-03 2.8021726e-01 1.4629277e-02 5.0365958e-02]
 [2.4232481e+00 3.1293817e-03 2.6073055e-02 4.5566950e+00]]


### Task 2
Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero

In [15]:
# рандомная матрица
numpy.random.seed(1729)
matrix = numpy.random.randn(4, 4)
matrix = matrix.astype(numpy.float32)

# Выделяем память на GPU и копируем матрицу на GPU
matrix_gpu = cuda.mem_alloc(matrix.nbytes)
cuda.memcpy_htod(matrix_gpu, matrix)

# CUDA-ядро для нахождения такой матрицы
mod_zero = SourceModule("""
__global__ void find_zero_matrix(float *matrix)
{
    int idx_x = threadIdx.x + blockIdx.x * blockDim.x;
    int idx_y = threadIdx.y + blockIdx.y * blockDim.y;
    int idx = idx_y * blockDim.x * gridDim.x + idx_x;

    matrix[idx] = -matrix[idx]; // Просто инвертируем каждый элемент
}
""")

zero_func = mod_zero.get_function("find_zero_matrix")  # функция кернела

# кернел делаем размером блока (4, 4, 1) и сетку размером (1, 1)
block_size = (4, 4, 1)
grid_size = (1, 1)
zero_func(matrix_gpu, block=block_size, grid=grid_size)

# копируем результат обратно на хост
result = numpy.empty_like(matrix)
cuda.memcpy_dtoh(result, matrix_gpu)

print("Original matrix:")
print(matrix)
print("\nMatrix to add to get zero matrix:")
print(result)

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


Original matrix:
[[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]

Matrix to add to get zero matrix:
[[ 0.6873394   0.82099473 -1.6523609   0.57529306]
 [-1.0989678  -0.92594606  0.9934138   0.8582211 ]
 [-0.07488676 -0.5293555  -0.12095155  0.22442362]
 [ 1.5566785  -0.05594088 -0.16147153  2.1346416 ]]


### Task 3

Write a cuda kernel to multiply two matrices:

1. Assume square matrices, with dimensions < 1024
2. Assume square matrices, with dimensions > 1024
3. Assume non-square matrices, with dimensions > 1024

In [16]:
# Assume square matrices, with dimensions < 1024

def generate_matrix(rows, cols):
    return numpy.random.randn(rows, cols).astype(numpy.float32)

# CUDA-ядро для умножения двух матриц
mod_mult = SourceModule("""
__global__ void matrix_multiply(float *a, float *b, float *c, int N)
{
    int idx_x = threadIdx.x + blockIdx.x * blockDim.x;
    int idx_y = threadIdx.y + blockIdx.y * blockDim.y;

    if (idx_x < N && idx_y < N) {
        float val = 0.0f;
        for (int k = 0; k < N; ++k) {
            val += a[idx_y * N + k] * b[k * N + idx_x];
        }
        c[idx_y * N + idx_x] = val;
    }
}
""")

mult_func = mod_mult.get_function("matrix_multiply")

# размер матрицы
size_small = 4

a_small = generate_matrix(size_small, size_small)
b_small = generate_matrix(size_small, size_small)

a_small_gpu = cuda.mem_alloc(a_small.nbytes)
b_small_gpu = cuda.mem_alloc(b_small.nbytes)

cuda.memcpy_htod(a_small_gpu, a_small)
cuda.memcpy_htod(b_small_gpu, b_small)

c_small_gpu = cuda.mem_alloc(a_small.nbytes)

block_size = (2, 2, 1)
grid_size = (size_small // block_size[0] + 1, size_small // block_size[1] + 1)
mult_func(a_small_gpu, b_small_gpu, c_small_gpu, numpy.int32(size_small), block=block_size, grid=grid_size)

c_small_result = numpy.empty_like(a_small)
cuda.memcpy_dtoh(c_small_result, c_small_gpu)
print("Small matrix multiplication result:")
print(c_small_result)

Small matrix multiplication result:
[[ 0.7838821  -0.90320694 -0.40178114  0.35484213]
 [-3.324403    2.613077    3.0019403  -1.3219621 ]
 [ 0.5503916   0.44426364 -1.0097607   0.4727013 ]
 [ 0.20731163  2.0331752  -1.9710609  -4.3500133 ]]


In [17]:
# Assume square matrices, with dimensions > 1024

def generate_matrix(rows, cols):
    return numpy.random.randn(rows, cols).astype(numpy.float32)

# CUDA-ядро для умножения двух матриц
mod_mult = SourceModule("""
__global__ void matrix_multiply(float *a, float *b, float *c, int N)
{
    int idx_x = threadIdx.x + blockIdx.x * blockDim.x;
    int idx_y = threadIdx.y + blockIdx.y * blockDim.y;

    if (idx_x < N && idx_y < N) {
        float val = 0.0f;
        for (int k = 0; k < N; ++k) {
            val += a[idx_y * N + k] * b[k * N + idx_x];
        }
        c[idx_y * N + idx_x] = val;
    }
}
""")

mult_func = mod_mult.get_function("matrix_multiply")

# размер матриц
size_large = 1030

a_large = generate_matrix(size_large, size_large)
b_large = generate_matrix(size_large, size_large)

a_large_gpu = cuda.mem_alloc(a_large.nbytes)
b_large_gpu = cuda.mem_alloc(b_large.nbytes)

cuda.memcpy_htod(a_large_gpu, a_large)
cuda.memcpy_htod(b_large_gpu, b_large)

c_large_gpu = cuda.mem_alloc(a_large.nbytes)

block_size = (32, 32, 1)
grid_size = (size_large // block_size[0] + 1, size_large // block_size[1] + 1)
mult_func(a_large_gpu, b_large_gpu, c_large_gpu, numpy.int32(size_large), block=block_size, grid=grid_size)

c_large_result = numpy.empty_like(a_large)
cuda.memcpy_dtoh(c_large_result, c_large_gpu)
print("\nLarge square matrix multiplication result:")
print(c_large_result)


Large square matrix multiplication result:
[[ 19.105267   26.929924   18.521933  ...  46.614548  -52.872543
  -21.14205  ]
 [-15.012323   -0.8235741  35.496998  ...  19.907202   -7.8865356
   67.96503  ]
 [ 17.873846  -32.292686    1.0756705 ... -33.64744    19.813179
   46.307983 ]
 ...
 [ 37.80699   -10.161602  -37.41534   ... -11.108911   14.936888
  -14.925332 ]
 [-21.287693   22.055311   35.86268   ...  99.76683   -43.925674
  -54.055645 ]
 [ -9.169684   48.606525   43.21028   ... -27.21785    10.442835
  -60.03849  ]]


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


In [18]:
# Assume non-square matrices, with dimensions > 1024

def generate_matrix(rows, cols):
    return numpy.random.randn(rows, cols).astype(numpy.float32)

# CUDA-ядро для умножения двух матриц
mod_mult = SourceModule("""
__global__ void matrix_multiply(float *a, float *b, float *c, int N)
{
    int idx_x = threadIdx.x + blockIdx.x * blockDim.x;
    int idx_y = threadIdx.y + blockIdx.y * blockDim.y;

    if (idx_x < N && idx_y < N) {
        float val = 0.0f;
        for (int k = 0; k < N; ++k) {
            val += a[idx_y * N + k] * b[k * N + idx_x];
        }
        c[idx_y * N + idx_x] = val;
    }
}
""")

mult_func = mod_mult.get_function("matrix_multiply")

# задаём размеры матриц
size_rows_large = 1050
size_cols_large = 1070

a_rect = generate_matrix(size_rows_large, size_cols_large)
b_rect = generate_matrix(size_cols_large, size_rows_large)

a_rect_gpu = cuda.mem_alloc(a_rect.nbytes)
b_rect_gpu = cuda.mem_alloc(b_rect.nbytes)

cuda.memcpy_htod(a_rect_gpu, a_rect)
cuda.memcpy_htod(b_rect_gpu, b_rect)

# выделяем память под результат на GPU
c_rect_gpu = cuda.mem_alloc(a_rect.nbytes)

block_size = (32, 32, 1)
grid_size_rect = (size_cols_large // block_size[0] + 1, size_rows_large // block_size[1] + 1)
mult_func(a_rect_gpu, b_rect_gpu, c_rect_gpu, numpy.int32(size_cols_large), block=block_size, grid=grid_size_rect)

c_rect_result = numpy.empty_like(a_rect)
cuda.memcpy_dtoh(c_rect_result, c_rect_gpu)
print("\nLarge rectangular matrix multiplication result:")
print(c_rect_result)


Large rectangular matrix multiplication result:
[[  -6.003228    -8.127362    -5.2243066 ...   -3.6504724   -6.505844
   -35.203945 ]
 [ -21.96266     -2.4735224  -24.543442  ...    3.784975   -44.2885
    26.5535   ]
 [   9.510945    -1.0790826   35.151436  ... -100.9466       9.979017
   -35.656456 ]
 ...
 [ -37.479874    16.72282    -68.070984  ...  -11.42971     13.359587
   -69.39042  ]
 [  10.844261   -28.928165   -48.18363   ...   38.748394   -12.242909
    29.673944 ]
 [  -3.8384736    0.5997028   17.338821  ...  -23.069664    44.882378
   -10.977278 ]]


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
